In [2]:
import json

from tqdm.auto import tqdm
import requests
import pandas as pd
from docx import Document


client_cert = "20250129_IDU_ADGX_LeonTur.crt"
ca_cert = "onti-ca.crt"
client_key = "DECFILE"

In [13]:
def construct_prompt(context):
    """
    Формирует промпт для модели на основе контекста.
    """

    context_str = "\n".join(context) if isinstance(context, list) else str(context)
    dict_example = {
        "name": "Юбилейный",
        "location": "Дубровка, Россия",
    }
    prompt = f"""
        Найди названия в тексте {context_str}.
        Названия должны принадлежать населеённым пунктам или административным единицам, которые физически представлены на территории.
        Приведи названия в начальную форму. Добавь как можно больше названий на основе контекста. В качестве ответа напиши только название. Если название не представлено в тексте или текст отсутствует, в качестве ответа дай ''
        Если названий больше одного, сохрани названия элементами в списке через запятую [Название1, Название2, Название3]
        """
    return prompt

In [14]:
def request_from_llm(prompt: str) -> str:
    """
    Function extracts prompt from llm api
    Args:
        prompt (str): prompt to execute
    Returns:
        str: request from llm
    """

    headers = {
        "Content-Type": "application/json"
    }
    data = {
        "model": "llama3.3",
        "prompt": prompt,
        "stream": False,
    }
    response = requests.post(
        url="http://a.dgx:11434/api/generate",
        headers=headers,
        data=json.dumps(data),
        cert=(client_cert, client_key),
        verify=ca_cert
    )
    if response.status_code == 200:
        return response.json()["response"]
    return ""

In [15]:
def extract_ner(text: str) -> str:

    prompt = construct_prompt(text)
    response_text = request_from_llm(prompt)
    return response_text
test = extract_ner("В Санкт-Петерубрге сегодня несолнечно, холодно, депрессивно и вообще повеситься хочется")
test

'Санкт-Петербург'

In [16]:
test_doc = Document("стратегия_2035.docx")

In [ ]:
list_for_save = []
current_par = []
switcher = False
privios_par = None
total_parapgraphs = []
for paragraph in tqdm(test_doc.paragraphs, desc="Processing texts"):
    if paragraph.style.name == "ConsPlusTitle":
        if switcher:
            current_par = [paragraph.text]
            total_parapgraphs.append(paragraph.text)
            switcher = False
        else:
            current_par.append(paragraph.text)
            total_parapgraphs.append(paragraph.text)
    else:
        if not switcher:
            switcher = True
        if paragraph.text:
            list_for_save.append(
                {
                    "text": paragraph.text,
                    "title": "".join(current_par),
                    "NER": extract_ner(paragraph.text),
                }
            )

Processing texts:   0%|          | 0/1493 [00:00<?, ?it/s]

In [10]:
df = pd.DataFrame.from_records(list_for_save)

In [11]:
df

,text,title,NER
0,ПРИЛОЖЕНИЕ,,ПРИЛОЖЕНИЕ
1,к Закону Санкт-Петербурга,,Не предоставлен текст к Закону Санкт-Петербурга.
2,"""О Стратегии",,"Поскольку текст ""О Стратегии"" не предоставлен,..."
3,социально-экономического,,"Не вижу предоставленного текста. Пожалуйста, д..."
4,развития Санкт-Петербурга,,"Петербург, Ленинградская область"
...,...,...,...
1398,Перечень государственных программ Санкт-Петерб...,11. Информация о государственных программахСан...,Санкт-Петербург
1399,,11. Информация о государственных программахСан...,"Пожалуйста, предоставьте текст для поиска назв..."
1400,,11. Информация о государственных программахСан...,Поскольку вы не предоставили текст для анализа...
1401,,11. Информация о государственных программахСан...,"Пожалуйста, предоставьте текст для анализа. Я ..."


In [32]:
name = " ".join(df[df["title"].isin([""])]["text"].to_list())

In [33]:
name

'ПРИЛОЖЕНИЕ к Закону Санкт-Петербурга "О Стратегии социально-экономического развития Санкт-Петербурга на период до 2035 года" от 19.12.2018 N 771-164 '